In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Cihazı ayarlayın (GPU varsa CUDA, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

# Verileri yükle
file_path_first = "C:/Users/alica/Desktop/10k_data_first.txt"
file_path_second = "C:/Users/alica/Desktop/10k_data_second.txt"

# İlk veri setini yükle
first_data = pd.read_csv(
    file_path_first,
    delimiter="_",
    names=["M", "Peri", "Node", "Incl.", "E", "N", "A", "Epoch", "x", "y", "z"]
)

# İkinci veri setini yükle (2026 tarihindeki gerçek konumlar)
second_data = pd.read_csv(
    file_path_second,
    delimiter="_",
    names=["x_2026", "y_2026", "z_2026"]
)

# Epoch tarihini `days_since_epoch` olarak hesaplayın
first_data["Epoch"] = pd.to_datetime(first_data["Epoch"], format='%Y%m%d')
target_date = pd.Timestamp("2026-01-01")
first_data["days_since_epoch"] = (target_date - first_data["Epoch"]).dt.days

# Özellikleri ve hedef değerleri ayır
X = first_data[["M", "Peri", "Node", "Incl.", "E", "N", "A", "days_since_epoch"]].values
y_2026 = second_data[["x_2026", "y_2026", "z_2026"]].values  # 2026 hedef konumlar

# Eğitim ve test setlerine bölme
X_train, X_val, y_train, y_val = train_test_split(X, y_2026, test_size=0.2, random_state=42)

# Veriyi ölçeklendir
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_val = scaler_X.transform(X_val)

y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)

# PyTorch tensörlerine dönüştür ve GPU'ya taşı
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# PyTorch DataLoader ile veri kümelerini oluştur
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Modeli oluştur
class PositionNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(PositionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, output_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = PositionNN(input_size=X_train.shape[1], output_size=3).to(device)

# Optimizasyon ve düzenlileştirme
criterion = nn.SmoothL1Loss()  # Daha stabil kayıp fonksiyonu
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Eğitim döngüsü
epochs = 100
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Doğrulama seti üzerinde değerlendirme
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
    val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Tüm veri üzerinde tahmin yap
X_tensor = torch.tensor(scaler_X.transform(X), dtype=torch.float32).to(device)
with torch.no_grad():
    predictions_scaled = model(X_tensor)
predictions = scaler_y.inverse_transform(predictions_scaled.cpu().numpy())

# Ortalama hata hesaplama
errors = np.abs(predictions - y_2026)
mean_error = np.mean(errors)
print(f"Ortalama hata (AU): {mean_error:.4f}")

# Ortalama gerçek değer büyüklüğü
mean_true_magnitude = np.mean(np.abs(y_2026))



# Tahminleri ve gerçek değerleri karşılaştırma
comparison = pd.DataFrame({
    "x_pred": predictions[:, 0],
    "y_pred": predictions[:, 1],
    "z_pred": predictions[:, 2],
    "x_true": y_2026[:, 0],
    "y_true": y_2026[:, 1],
    "z_true": y_2026[:, 2]
})

# Sonuçları CSV formatında kaydet
comparison.to_csv("C:/Users/alica/Desktop/comparison_results_fixed.csv", index=False)
print("Sonuçlar 'comparison_results_fixed.csv' dosyasına kaydedildi.")


Yukarıdaki blok sayesınde yapay zeka %15 - %20 doğruluk oranı ile istenilen veriyi hesaplamaktadır.

In [ ]:
import pandas as pd

# Dosya yolu ve çıktı dosyası
file_path = "C:/Users/alica/Desktop/comparison_results_fixed.csv"  # Giriş dosyasının yolu
output_file = "C:/Users/alica/Desktop/caglar.csv"  # Çıktı dosyasının adı

# Verileri yükle
data = pd.read_csv(file_path)

# Farkları hesapla ve yüzdelik farkları bul
data['x_diff'] = ((data['x_pred'] - data['x_true']) / data['x_true']) * 100
data['y_diff'] = ((data['y_pred'] - data['y_true']) / data['y_true']) * 100
data['z_diff'] = ((data['z_pred'] - data['z_true']) / data['z_true']) * 100

# NaN ve sonsuz değerleri temizle
data['x_diff'] = data['x_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)
data['y_diff'] = data['y_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)
data['z_diff'] = data['z_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)

# Ortalama yüzdelik fark hesaplama
data['average_percent'] = (
    data[['x_diff', 'y_diff', 'z_diff']].mean(axis=1)
)

# %70'ten büyük hata paylarını tespit et
high_error_rows = data[data['average_percent'].abs() > 70]
filtered_data = data[data['average_percent'].abs() <= 70]  # Genel hata için filtrelenmiş veriler

# %70'ten büyük hata paylarının sayısını bul
high_error_count = len(high_error_rows)
print(f"%70'den büyük hata payına sahip {high_error_count} satır bulundu.")

# Genel ortalama hata yüzdesini filtrelenmiş verilerle hesapla
if len(filtered_data) > 0:
    total_average_error = (
        filtered_data[['x_diff', 'y_diff', 'z_diff']].abs().sum().sum() / (len(filtered_data) * 3)
    )
    print(f"Genel Ortalama Hata Payı (filtrelenmiş): {total_average_error:.1f}%")
else:
    print("Filtrelenmiş veri bulunmamaktadır, genel hata payı hesaplanamadı.")

# Ortalama yüzdelik farkları tam sayı ve bir ondalık hane olacak şekilde yuvarla
data['average_percent'] = data['average_percent'].round(1).astype(str) + "%"

# Sadece `average_percent` sütununu kaydet
average_percentage = data[['average_percent']]
average_percentage.to_csv(output_file, index=False, header=False)

print(f"Sadece ortalama yüzdelik farklarla dosya '{output_file}' olarak kaydedildi.")


Yukarıdaki kod ile yapay zekanın yüzde kaç doğru eğitildiğini ve sonuclarnın yüzde kaç dogru oldugnu hesaplıyoruz